In [ ]:
import sys
sys.path.append('..')


In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from gerumo.data.generators import build_generator
from gerumo.data.dataset import describe_dataset
from gerumo.utils.engine import (
    setup_cfg, setup_environment, build_dataset
)
from gerumo.data.tfrecords import generator_to_record

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
args = dotdict()

## Select configuration

In [ ]:
args['config_file'] = '/home/asuka/projects/gerumo2/config/regression/cnn/cnn_lst.yml'
args['output_dir'] = './tfRecords/lst'
args['samples_per_file'] = 1000

args['opts'] = [
    'SOLVER.BATCH_SIZE', 1,
    'DATASETS.TRAIN.EVENTS', '/home/asuka/projects/gerumo2/datasets/gamma_sample/splits/validation_events.parquet',
    'DATASETS.TRAIN.TELESCOPES', '/home/asuka/projects/gerumo2/datasets/gamma_sample/splits/validation_telescopes.parquet'
]

## Setup

In [ ]:
cfg = setup_cfg(args)
logger = setup_environment(cfg)
output_dir = args.output_dir
samples_per_file = args.samples_per_file

## Load Datasets

In [ ]:
train_dataset = build_dataset(cfg, 'train')
validation_dataset = build_dataset(cfg, 'validation')

## Build generators

In [ ]:
train_generator = build_generator(cfg, train_dataset)
validation_generator = build_generator(cfg, validation_dataset)
input_shape = train_generator.get_input_shape()

In [ ]:
train_generator.fit_mode()
validation_generator.fit_mode()

In [ ]:
dataset_split = 'train'
dataset_generator = train_generator

describe_dataset(dataset_generator, logger, save_to=output_dir / f'{dataset_split}_description.txt')
generator_to_record(dataset_generator, dataset_split, output_dir, samples_per_file, input_shape)


In [ ]:
dataset_split = 'validation'
dataset_generator = validation_generator

describe_dataset(dataset_generator, logger, save_to=output_dir / f'{dataset_split}_description.txt')
generator_to_record(dataset_generator, dataset_split, output_dir, samples_per_file, input_shape)